In [ ]:
#ライブラリのインポート
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

## GPUによる高速化

TensorflowやPyTorchといった深層学習ライブラリはGPUでの実行がサポートされています。
GPUを使うことで大規模計算が高速化できることがあります。

このノートブックではTensorflow/KerasやPyTorchをGPU上で実行し、CPUでの実行したときと計算速度を比較します。


### GPUの利用状況の確認

現在接続しているノードのGPU使用率は以下のコマンドで確認できます。
```bash
$ nvidia-smi
Thu Jul 20 11:29:18 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 520.61.05    Driver Version: 520.61.05    CUDA Version: 11.8     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-PCI...  On   | 00000000:1F:00.0 Off |                    0 |
| N/A   84C    P0   129W / 250W |   3493MiB / 40960MiB |     78%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA A100-PCI...  On   | 00000000:20:00.0 Off |                    0 |
| N/A   84C    P0   175W / 250W |   2823MiB / 40960MiB |     79%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+

+-----------------------------------------------------------------------------+
| Processes:                                                                  |
|  GPU   GI   CI        PID   Type   Process name                  GPU Memory |
|        ID   ID                                                   Usage      |
|=============================================================================|
|    0   N/A  N/A   3460006      C   python                           1242MiB |
|    0   N/A  N/A   3461170      C   ...HostWorker.run_executable     1494MiB |
|    1   N/A  N/A   3460006      C   python                            598MiB |
|    1   N/A  N/A   3461170      C   ...HostWorker.run_executable     2216MiB |
+-----------------------------------------------------------------------------+
```
このノードにはGPUが2台搭載されており、それぞれ2つのプロセスがGPUを使用しています。

自分のプロセスがゾンビ状態となってGPUのメモリを専有してしまうことがないように定期的にチェックするのが好ましいです。

GPUを使っているプロセスを起動したユーザーを確認するコマンドの一例は以下です。
```bash
$ ps -up `nvidia-smi --query-compute-apps=pid --format=csv,noheader | sort -u`
USER         PID %CPU %MEM    VSZ   RSS TTY      STAT START   TIME COMMAND
```
ここで表示されるメモリの値はGPU上のメモリではないことに注意してください。

notebook上では"!"をつけることでbashコマンドが実行できます。     

In [ ]:
!nvidia-smi

In [ ]:
!ps -up `nvidia-smi --query-compute-apps=pid --format=csv,noheader | sort -u`

### 使用するGPUの指定
まずは使うGPUを指定します。このステップなしでもGPUは利用できますが、GPUを専有してしまうことで共有マシンを使用している他のユーザーに迷惑をかけてしまうことがあります。
今回の講習では、使うGPUは1つのみに限定させます。

In [ ]:
def get_available_gpu_index():
    import random
    import subprocess

    # ノード上のGPUのインデックスとuuidを取得します。
    p = subprocess.run(
        f"nvidia-smi --query-gpu=index,gpu_uuid --format=csv,noheader",
        shell=True,
        stdout=subprocess.PIPE,
    )
    gpu_uuid_to_index = {}
    for v in p.stdout.decode().split('\n'):
        if len(v) == 0:
            continue
        index, uuid = v.split(',')
        gpu_uuid_to_index[uuid.strip()] = index.strip()

    # GPUを使っているプロセスのuuidを取得します。
    p = subprocess.run(
        f"nvidia-smi --query-compute-apps=gpu_uuid --format=csv,noheader",
        shell=True,
        stdout=subprocess.PIPE,
    )
    ret = p.stdout.decode().split('\n')
    ret = set(ret)
    for uuid in ret:
        if len(uuid) == 0:
            continue
        gpu_uuid_to_index.pop(uuid)

    # どのプロセスからも使用されていないGPUのインデックスをランダムに一つ返します。
    # GPUが使用できない場合は空文字列を返します。
    if len(gpu_uuid_to_index) == 0:
        print('GPU is unavailable.')
        return ""
    else:
        available_gpu_index = list(gpu_uuid_to_index.values())
        return random.choice(available_gpu_index)

# 使用されていないGPUを選択します。
gpu_index = get_available_gpu_index()
print(gpu_index)

# 使うGPUを一つに制限します。
import os
os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES'] = gpu_index



### Tensorflow でのGPU利用の注意点

TensorflowはデフォルトでGPU上のメモリを全て確保しようとします。GPUを利用するのが1人だけの場合はこれで良いのですが、複数人で共有する場合は他のユーザーのプロセスを停止させてしまうこともあります。
そのため、Tensorflowが必要な分だけGPUメモリを確保するように設定します。

In [ ]:
# Tensorflow
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action="ignore", category=DeprecationWarning)
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'
import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

# Tensorflowが使うCPUの数を制限します。(VMを使う場合)
os.environ['OMP_NUM_THREADS'] = '1'
os.environ['TF_NUM_INTEROP_THREADS'] = '1'
os.environ['TF_NUM_INTRAOP_THREADS'] = '1'

from tensorflow.config import threading
num_threads = 1
threading.set_inter_op_parallelism_threads(num_threads)
threading.set_intra_op_parallelism_threads(num_threads)

# GPUのメモリを使いすぎないように制限します。
physical_devices = tf.config.list_physical_devices('GPU')
if len(physical_devices) > 0:
    tf.config.set_visible_devices(physical_devices, 'GPU')
    for gpu in physical_devices:
        tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.list_logical_devices('GPU')
    print('available GPU:', logical_gpus)

## MLPのトレーニング

MLPのトレーニングをCPU/GPUでそれぞれ実行することで、計算時間の変化を確認します。

データセットは乱数で適当に作成します。

In [ ]:
from numpy.random import default_rng
rng = default_rng(seed=0)

# 10万行、入力次元10のランダムなデータを作成。
datasize = 100000
x = rng.normal(size=(datasize, 10))
t = rng.integers(0, 2, size=(datasize, 1))

## CPUによるサンプルコード

GPUがあるとGPUが自動で使われてしまいます。CPUで計算するように明示的に指定することでCPUで計算させることができます。

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

with tf.device('/CPU:0'):
    # モデルの定義
    model = Sequential([
        Dense(units=256, activation='relu', input_dim=10),  # ノード数が256の層を追加。活性化関数はReLU。
        Dense(units=256, activation='relu'),  # ノード数が256の層を追加。活性化関数はReLU。
        Dense(units=256, activation='relu'),  # ノード数が256の層を追加。活性化関数はReLU。
        Dense(units=256, activation='relu'),  # ノード数が256の層を追加。活性化関数はReLU。
        Dense(units=256, activation='relu'),  # ノード数が256の層を追加。活性化関数はReLU。
        Dense(units=1, activation='sigmoid')  # ノード数が1の層を追加。活性化関数はシグモイド関数。
    ])

    #  誤差関数としてクロスエントロピーを指定。最適化手法はadam
    model.compile(loss='binary_crossentropy', optimizer='adam')

    #  トレーニング
    model.fit(
        x=x,
        y=t,
        batch_size=1024,  # バッチサイズ。一回のステップで1024行のデータを使うようにする。
        epochs=10,  # 学習のステップ数
        verbose=1,  # 1とするとステップ毎に誤差関数の値などが表示される
    )



## GPUによるサンプルコード

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# モデルの定義
model = Sequential([
    Dense(units=256, activation='relu', input_dim=10),  # ノード数が256の層を追加。活性化関数はReLU。
    Dense(units=256, activation='relu'),  # ノード数が256の層を追加。活性化関数はReLU。
    Dense(units=256, activation='relu'),  # ノード数が256の層を追加。活性化関数はReLU。
    Dense(units=256, activation='relu'),  # ノード数が256の層を追加。活性化関数はReLU。
    Dense(units=256, activation='relu'),  # ノード数が256の層を追加。活性化関数はReLU。
    Dense(units=1, activation='sigmoid')  # ノード数が1の層を追加。活性化関数はシグモイド関数。
])

#  誤差関数としてクロスエントロピーを指定。最適化手法はadam
model.compile(loss='binary_crossentropy', optimizer='adam')

#  トレーニング
model.fit(
    x=x,
    y=t,
    batch_size=1024,  # バッチサイズ。一回のステップで1024行のデータを使うようにする。
    epochs=10,  # 学習のステップ数
    verbose=1,  # 1とするとステップ毎に誤差関数の値などが表示される
)



計算時間はどのように変化したでしょうか。
モデルサイズが変化すると、計算時間はどのように変わるでしょうか？